In [1]:
import cosmofast as cf
import numpy as np
f_tt = np.load('cosmofast/planck_18/data/plik_lite_tt.npz')
f_ttteee = np.load('cosmofast/planck_18/data/plik_lite_ttteee.npz')
test_tt = np.load("cosmofast/tests/test_plik_lite_tt.npz")
test_ttteee = np.load("cosmofast/tests/test_plik_lite_ttteee.npz")

In [2]:
cls_b = np.empty(f_ttteee['n_bin'])

# power[30:, 0], TT
cf.planck_18._plik_lite._get_binned_cls(
    test_ttteee['test_param'][30:2509],
    cls_b[f_ttteee['i_bin'][0]:f_ttteee['i_bin'][1]], f_ttteee['weight_t'],
    f_ttteee['low_t'], f_ttteee['width_t'], f_ttteee['n_bin_t'])

# power[30:, 3], TE
cf.planck_18._plik_lite._get_binned_cls(
    test_ttteee['test_param'][(2509+2509+30):(2509+2509+2509)],
    cls_b[f_ttteee['i_bin'][1]:f_ttteee['i_bin'][2]], f_ttteee['weight_e'],
    f_ttteee['low_e'], f_ttteee['width_e'], f_ttteee['n_bin_e'])

# power[30:, 1], EE
cf.planck_18._plik_lite._get_binned_cls(
    test_ttteee['test_param'][(2509+30):(2509+2509)],
    cls_b[f_ttteee['i_bin'][2]:f_ttteee['i_bin'][3]], f_ttteee['weight_e'],
    f_ttteee['low_e'], f_ttteee['width_e'], f_ttteee['n_bin_e'])

cls_b = cls_b @ f_ttteee['cov_inv_low']

In [3]:
test_f = np.empty(1)
cf.planck_18._plik_lite._plik_lite_f(
    cls_b, test_ttteee['test_param'][-1], test_f, f_ttteee['mu'],
    f_ttteee['n_bin'])
test_f, test_ttteee['test_value']

(array([-292.16906205]), array([-292.16906205]))

In [11]:
test_j = np.empty((1, f_ttteee['n_bin'] + 1))
cf.planck_18._plik_lite._plik_lite_j(
    cls_b, test_ttteee['test_param'][-1], test_j, f_ttteee['mu'],
    f_ttteee['n_bin'])
test_j[0, -100:]

array([-3.50284490e-01,  1.23750434e+00,  9.35106885e-01, -1.36290047e+00,
        6.30176628e-01,  4.78517676e-01,  1.21677280e+00,  9.57135167e-01,
       -3.97714159e-01, -1.25565418e-01, -3.17786720e-01, -7.71286841e-01,
       -1.74313485e+00, -1.88753694e-02, -5.14096761e-01,  2.10933428e+00,
        1.36420878e+00,  5.68018096e-01,  3.34293040e-01,  9.91503453e-01,
        2.13778428e+00, -3.10635107e-03, -3.66558017e-01, -8.55155719e-01,
        1.60236515e+00, -9.75822204e-01, -5.14578581e-01, -1.17549762e-01,
        3.53188480e-01, -3.81651337e-02,  9.50918959e-02,  2.87143373e-01,
       -1.35770063e+00, -1.09953330e+00, -7.46018863e-01, -9.30609151e-01,
        1.46387420e+00,  4.55375802e-01, -7.28239931e-01,  3.85085033e-01,
        6.81252120e-01,  7.33129397e-01,  2.70199773e-01,  7.81511115e-01,
       -3.44698486e-01, -9.87502192e-01,  5.18733329e-01,  4.93572271e-01,
        4.89098246e-01,  1.32534773e+00, -1.10432781e+00,  8.64640576e-01,
       -2.14754023e-01, -

In [12]:
test_fjf = np.empty(1)
test_fjj = np.empty((1, f_ttteee['n_bin'] + 1))
cf.planck_18._plik_lite._plik_lite_fj(
    cls_b, test_ttteee['test_param'][-1], test_fjf, test_fjj, f_ttteee['mu'],
    f_ttteee['n_bin'])
test_fjf, test_fjj[0, -100:]

(array([-292.16906205]),
 array([-3.50284490e-01,  1.23750434e+00,  9.35106885e-01, -1.36290047e+00,
         6.30176628e-01,  4.78517676e-01,  1.21677280e+00,  9.57135167e-01,
        -3.97714159e-01, -1.25565418e-01, -3.17786720e-01, -7.71286841e-01,
        -1.74313485e+00, -1.88753694e-02, -5.14096761e-01,  2.10933428e+00,
         1.36420878e+00,  5.68018096e-01,  3.34293040e-01,  9.91503453e-01,
         2.13778428e+00, -3.10635107e-03, -3.66558017e-01, -8.55155719e-01,
         1.60236515e+00, -9.75822204e-01, -5.14578581e-01, -1.17549762e-01,
         3.53188480e-01, -3.81651337e-02,  9.50918959e-02,  2.87143373e-01,
        -1.35770063e+00, -1.09953330e+00, -7.46018863e-01, -9.30609151e-01,
         1.46387420e+00,  4.55375802e-01, -7.28239931e-01,  3.85085033e-01,
         6.81252120e-01,  7.33129397e-01,  2.70199773e-01,  7.81511115e-01,
        -3.44698486e-01, -9.87502192e-01,  5.18733329e-01,  4.93572271e-01,
         4.89098246e-01,  1.32534773e+00, -1.10432781e+00,  8.6

In [15]:
import numdifftools as nd

xx = np.concatenate((cls_b, [test_ttteee['test_param'][-1]]))

def bf_f(x):
    test_f = np.empty(1)
    cf.planck_18._plik_lite._plik_lite_f(
        x[:-1], x[-1], test_f, f_ttteee['mu'],
        f_ttteee['n_bin'])
    return test_f

def bf_j(x):
    test_j = np.empty((1, f_ttteee['n_bin'] + 1))
    cf.planck_18._plik_lite._plik_lite_j(
        x[:-1], x[-1], test_j, f_ttteee['mu'],
        f_ttteee['n_bin'])
    return test_j

bf_f(xx), bf_j(xx)[0, -100:]

(array([-292.16906205]),
 array([-3.50284490e-01,  1.23750434e+00,  9.35106885e-01, -1.36290047e+00,
         6.30176628e-01,  4.78517676e-01,  1.21677280e+00,  9.57135167e-01,
        -3.97714159e-01, -1.25565418e-01, -3.17786720e-01, -7.71286841e-01,
        -1.74313485e+00, -1.88753694e-02, -5.14096761e-01,  2.10933428e+00,
         1.36420878e+00,  5.68018096e-01,  3.34293040e-01,  9.91503453e-01,
         2.13778428e+00, -3.10635107e-03, -3.66558017e-01, -8.55155719e-01,
         1.60236515e+00, -9.75822204e-01, -5.14578581e-01, -1.17549762e-01,
         3.53188480e-01, -3.81651337e-02,  9.50918959e-02,  2.87143373e-01,
        -1.35770063e+00, -1.09953330e+00, -7.46018863e-01, -9.30609151e-01,
         1.46387420e+00,  4.55375802e-01, -7.28239931e-01,  3.85085033e-01,
         6.81252120e-01,  7.33129397e-01,  2.70199773e-01,  7.81511115e-01,
        -3.44698486e-01, -9.87502192e-01,  5.18733329e-01,  4.93572271e-01,
         4.89098246e-01,  1.32534773e+00, -1.10432781e+00,  8.6

In [17]:
nd.Gradient(bf_f, step=1e-8)(xx)[-100:]

array([-3.50286202e-01,  1.23750397e+00,  9.35108352e-01, -1.36290056e+00,
        6.30177514e-01,  4.78519272e-01,  1.21677318e+00,  9.57135177e-01,
       -3.97713509e-01, -1.25564271e-01, -3.17785977e-01, -7.71285618e-01,
       -1.74313471e+00, -1.88748573e-02, -5.14097568e-01,  2.10933423e+00,
        1.36420795e+00,  5.68016393e-01,  3.34293306e-01,  9.91505550e-01,
        2.13778436e+00, -3.10649286e-03, -3.66557631e-01, -8.55158083e-01,
        1.60236483e+00, -9.75822451e-01, -5.14577895e-01, -1.17549349e-01,
        3.53188059e-01, -3.81646717e-02,  9.50905138e-02,  2.87144532e-01,
       -1.35770223e+00, -1.09953363e+00, -7.46018718e-01, -9.30609195e-01,
        1.46387436e+00,  4.55375473e-01, -7.28240939e-01,  3.85085743e-01,
        6.81251326e-01,  7.33129473e-01,  2.70199509e-01,  7.81511749e-01,
       -3.44698494e-01, -9.87500931e-01,  5.18733148e-01,  4.93571409e-01,
        4.89097832e-01,  1.32534695e+00, -1.10432837e+00,  8.64642407e-01,
       -2.14754438e-01, -

In [19]:
cls_b = np.empty(f_tt['n_bin'])

# power[30:, 0]
cf.planck_18._plik_lite._get_binned_cls(
    test_tt['test_param'][30:2509], cls_b, f_tt['weight_t'], f_tt['low_t'],
    f_tt['width_t'], f_tt['n_bin_t'])

cls_b = cls_b @ f_tt['cov_inv_low']

In [20]:
test_f = np.empty(1)
cf.planck_18._plik_lite._plik_lite_f(
    cls_b, test_tt['test_param'][-1], test_f, f_tt['mu'], f_tt['n_bin'])
test_f, test_tt['test_value']

(array([-103.46972499]), array([-103.46972499]))

In [21]:
test_j = np.empty((1, f_tt['n_bin'] + 1))
cf.planck_18._plik_lite._plik_lite_j(
    cls_b, test_tt['test_param'][-1], test_j, f_tt['mu'],
    f_tt['n_bin'])
test_j[0, -100:]

array([-7.22996602e-01, -6.71835237e-01, -5.42342453e-01, -8.63347740e-02,
        1.37079871e+00, -5.51169586e-01, -1.02468352e-01,  2.20618163e+00,
        8.55425843e-01, -4.62789408e-01, -9.80969750e-01,  4.43005089e-01,
       -1.16434205e-01, -1.03470343e+00, -4.89031559e-01,  9.52509631e-01,
       -1.20153405e+00, -3.54974017e-01,  1.23379573e+00,  2.61019794e-01,
       -2.49862503e+00, -1.74826629e+00,  6.79403797e-01, -1.01186184e+00,
        1.01809047e+00,  9.21172094e-01, -9.44582619e-01, -9.65122841e-01,
        3.56434746e-01, -8.53669841e-01,  2.25918871e+00,  7.69849040e-01,
       -8.17774164e-01, -2.18526094e-01, -5.78426954e-02,  2.33212130e+00,
       -3.56242041e-01,  1.09393208e+00,  9.89615262e-01, -1.13600766e-01,
       -1.34660533e+00,  5.95601238e-01,  1.85762397e+00,  6.44363703e-01,
       -4.75518783e-02, -1.78993530e+00, -8.65657419e-01, -6.91312211e-01,
       -1.86051498e+00, -1.59965108e+00, -1.11734469e+00, -2.19610229e-01,
        9.36701855e-01,  

In [22]:
test_fjf = np.empty(1)
test_fjj = np.empty((1, f_tt['n_bin'] + 1))
cf.planck_18._plik_lite._plik_lite_fj(
    cls_b, test_tt['test_param'][-1], test_fjf, test_fjj, f_tt['mu'],
    f_tt['n_bin'])
test_fjf, test_fjj[0, -100:]

(array([-103.46972499]),
 array([-7.22996602e-01, -6.71835237e-01, -5.42342453e-01, -8.63347740e-02,
         1.37079871e+00, -5.51169586e-01, -1.02468352e-01,  2.20618163e+00,
         8.55425843e-01, -4.62789408e-01, -9.80969750e-01,  4.43005089e-01,
        -1.16434205e-01, -1.03470343e+00, -4.89031559e-01,  9.52509631e-01,
        -1.20153405e+00, -3.54974017e-01,  1.23379573e+00,  2.61019794e-01,
        -2.49862503e+00, -1.74826629e+00,  6.79403797e-01, -1.01186184e+00,
         1.01809047e+00,  9.21172094e-01, -9.44582619e-01, -9.65122841e-01,
         3.56434746e-01, -8.53669841e-01,  2.25918871e+00,  7.69849040e-01,
        -8.17774164e-01, -2.18526094e-01, -5.78426954e-02,  2.33212130e+00,
        -3.56242041e-01,  1.09393208e+00,  9.89615262e-01, -1.13600766e-01,
        -1.34660533e+00,  5.95601238e-01,  1.85762397e+00,  6.44363703e-01,
        -4.75518783e-02, -1.78993530e+00, -8.65657419e-01, -6.91312211e-01,
        -1.86051498e+00, -1.59965108e+00, -1.11734469e+00, -2.1

In [23]:
import numdifftools as nd

xx = np.concatenate((cls_b, [test_tt['test_param'][-1]]))

def bf_f(x):
    test_f = np.empty(1)
    cf.planck_18._plik_lite._plik_lite_f(
        x[:-1], x[-1], test_f, f_tt['mu'],
        f_tt['n_bin'])
    return test_f

def bf_j(x):
    test_j = np.empty((1, f_tt['n_bin'] + 1))
    cf.planck_18._plik_lite._plik_lite_j(
        x[:-1], x[-1], test_j, f_tt['mu'],
        f_tt['n_bin'])
    return test_j

bf_f(xx), bf_j(xx)[0, -100:]

(array([-103.46972499]),
 array([-7.22996602e-01, -6.71835237e-01, -5.42342453e-01, -8.63347740e-02,
         1.37079871e+00, -5.51169586e-01, -1.02468352e-01,  2.20618163e+00,
         8.55425843e-01, -4.62789408e-01, -9.80969750e-01,  4.43005089e-01,
        -1.16434205e-01, -1.03470343e+00, -4.89031559e-01,  9.52509631e-01,
        -1.20153405e+00, -3.54974017e-01,  1.23379573e+00,  2.61019794e-01,
        -2.49862503e+00, -1.74826629e+00,  6.79403797e-01, -1.01186184e+00,
         1.01809047e+00,  9.21172094e-01, -9.44582619e-01, -9.65122841e-01,
         3.56434746e-01, -8.53669841e-01,  2.25918871e+00,  7.69849040e-01,
        -8.17774164e-01, -2.18526094e-01, -5.78426954e-02,  2.33212130e+00,
        -3.56242041e-01,  1.09393208e+00,  9.89615262e-01, -1.13600766e-01,
        -1.34660533e+00,  5.95601238e-01,  1.85762397e+00,  6.44363703e-01,
        -4.75518783e-02, -1.78993530e+00, -8.65657419e-01, -6.91312211e-01,
        -1.86051498e+00, -1.59965108e+00, -1.11734469e+00, -2.1

In [24]:
nd.Gradient(bf_f, step=1e-8)(xx)[-100:]

array([-7.22996423e-01, -6.71835214e-01, -5.42342352e-01, -8.63344956e-02,
        1.37079966e+00, -5.51169425e-01, -1.02468079e-01,  2.20618121e+00,
        8.55425957e-01, -4.62789276e-01, -9.80970333e-01,  4.43004924e-01,
       -1.16433797e-01, -1.03470371e+00, -4.89031041e-01,  9.52509544e-01,
       -1.20153346e+00, -3.54974363e-01,  1.23379565e+00,  2.61020007e-01,
       -2.49862568e+00, -1.74826625e+00,  6.79403915e-01, -1.01186189e+00,
        1.01809051e+00,  9.21172477e-01, -9.44582729e-01, -9.65122388e-01,
        3.56435239e-01, -8.53670206e-01,  2.25918983e+00,  7.69849611e-01,
       -8.17774298e-01, -2.18525999e-01, -5.78424423e-02,  2.33212135e+00,
       -3.56242682e-01,  1.09393171e+00,  9.89616217e-01, -1.13600863e-01,
       -1.34660639e+00,  5.95601083e-01,  1.85762447e+00,  6.44363500e-01,
       -4.75516518e-02, -1.78993603e+00, -8.65657062e-01, -6.91311901e-01,
       -1.86051566e+00, -1.59965127e+00, -1.11734480e+00, -2.19609576e-01,
        9.36701389e-01,  